# Import the packages to be used to analyze data
1. pandas: handles time-series data (weather data)
2. numpy: math operations
3. pvlib: solar energy modeling (sun position, irradiance, temperature)
4. scikit-learn: for splitting data and training ML models
5. matplotlib: for making scatter plots

In [2]:
import pandas as pd
import numpy as np
import pvlib
from pvlib import location, irradiance, temperature, pvsystem, modelchain
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


# Functions

1. read_weather(): Reads the weather CSV and sets up the datetime index.

In [3]:
def read_weather(csv_path, tz='UTC'):
    df = pd.read_csv(csv_path, parse_dates=[0], index_col=0)
    df.index = df.index.tz_localize(None).tz_localize(tz)
    # normalize column names
    df = df.rename(columns={c.lower(): c for c in df.columns})
    return df

2. ensure_frequency(): Makes sure the data is evenly spaced in time (e.g., every 15 minutes).

In [4]:
def ensure_frequency(df, freq='15T'):
    df = df.sort_index()
    df = df.resample(freq).interpolate(limit_direction='both')
    return df

3. compute_solar_geometry(): Computes the sun’s position at each timestamp.

In [5]:
def compute_solar_geometry(df, latitude, longitude, altitude=0, tz='UTC'):
    loc = location.Location(latitude, longitude, tz=tz, altitude=altitude)
    sp = loc.get_solarposition(df.index)
    # add zenith and azimuth
    df['solar_zenith'] = sp['zenith']
    df['solar_azimuth'] = sp['azimuth']
    return df

4. compute_poa(): Computes plane-of-array (POA) irradiance — how much sunlight hits the tilted panels.

In [6]:
def compute_poa(df, tilt, azimuth, albedo=0.2, model='perez'):
# if dni or dhi missing, try to estimate via erbs decomposition
    if 'dni' not in df.columns or 'dhi' not in df.columns:
        ghi = df['ghi'].values
        solar_zenith = df['solar_zenith'].values
        dni, dhi = irradiance.erbs(ghi, solar_zenith, df.index)
        df['dni'] = dni
        df['dhi'] = dhi

    poa = irradiance.get_total_irradiance(
        surface_tilt=tilt,
        surface_azimuth=azimuth,
        solar_zenith=df['solar_zenith'],
        solar_azimuth=df['solar_azimuth'],
        dni=df['dni'],
        ghi=df['ghi'],
        dhi=df['dhi'],
        albedo=albedo,
        model=model)

    df['poa_global'] = poa['poa_global']
    df['poa_direct'] = poa['poa_direct']
    df['poa_diffuse'] = poa['poa_diffuse']
    return df

5. estimate_cell_temperature(): Estimates the temperature of the solar cells.

In [7]:
def estimate_cell_temperature(df, mounting='open_rack', noct=None):
    if noct is not None:
        # T_c = T_a + (E_poa / 800) * (NOCT - 20)
        df['cell_temp'] = df['temp_air'] + (df['poa_global'] / 800.0) * (noct - 20.0)
        return df

    # else use sapm cell temp model (requires wind speed, ambient temp, poa_global)
    try:
        # use default model parameters for mounting types via pvlib
        params = temperature.TemperatureModelParameters.from_sapm(mounting_type=mounting)
        df['cell_temp'] = temperature.sapm_cell(df['poa_global'], df['temp_air'], df['wind_speed'], **params)
    except Exception:
        # fallback to simple approximation
        df['cell_temp'] = df['temp_air'] + (df['poa_global'] / 800.0) * 20.0
    return df

6. clear_sky_index(): Compares actual sunlight to what would happen on a clear day.

In [8]:
def clear_sky_index(df, latitude, longitude, tz='UTC'):
    loc = location.Location(latitude, longitude, tz=tz)
    cs = loc.get_clearsky(df.index, model='ineichen')
    df['ghi_cs'] = cs['ghi']
    df['k_clear'] = df['ghi'] / (df['ghi_cs'] + 1e-9)
    df['k_clear'] = df['k_clear'].clip(0.0, 2.0)
    return df

7. simple_dc_power(): Estimates DC power output from the panels.

In [9]:
def simple_dc_power(df, system_kwp, gamma_p=-0.004, p_stc_per_kwp=1000.0):
# P_dc = P_stc * (E_poa / 1000) * (1 + gamma*(Tcell-25))
    p_stc_kw = system_kwp
    df['p_dc_est'] = p_stc_kw * (df['poa_global'] / p_stc_per_kwp) * (1.0 + gamma_p * (df['cell_temp'] - 25.0))
    df['p_dc_est'] = df['p_dc_est'].clip(lower=0.0)
    return df

8. simple_inverter_ac(): Converts DC power to AC power after the inverter.

In [10]:
def simple_inverter_ac(df, inverter_ac_rating_kw, eta_const=0.98, min_eta=0.90):
    df['p_ac_est'] = df['p_dc_est'] * eta_const
    # clip at inverter maximum
    df['p_ac_est'] = df['p_ac_est'].clip(upper=inverter_ac_rating_kw)
    # apply tiny floor for night
    df.loc[df['poa_global'] < 10, 'p_ac_est'] = 0.0
    return df

9. apply_system_losses(): Applies overall losses like soiling, wiring, mismatch, etc.

In [11]:
def apply_system_losses(df, loss_factors=None):
    if loss_factors is None:
        loss_factors = {'soiling': 0.01, 'wiring': 0.01, 'mismatch': 0.01, 'availability': 0.01}
    total_loss = 1.0
    for v in loss_factors.values():
        total_loss *= (1.0 - v)
    df['p_ac_est_net'] = df['p_ac_est'] * total_loss
    return df

10. time_features(): Adds calendar/time-based features for machine learning.

In [12]:
def time_features(df):
    idx = df.index
    df['hour'] = idx.hour
    df['minute'] = idx.minute
    df['dow'] = idx.weekday
    df['doy'] = idx.dayofyear
    # cyclic encodings
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24.0)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24.0)
    df['doy_sin'] = np.sin(2 * np.pi * df['doy'] / 365.25)
    df['doy_cos'] = np.cos(2 * np.pi * df['doy'] / 365.25)
    return df

11. lag_and_rolling_features(): Adds historical features for forecasting.

In [13]:
def lag_and_rolling_features(df, cols, lags=[1,2,3,4], windows=['1H','3H','6H']):
    freq = pd.infer_freq(df.index)
    if freq is None:
        # fallback: compute based on median diff
        step_t = (df.index.to_series().diff().median())
        freq = f"{int(step_t.total_seconds()/60)}T"

    # convert lags in steps
    for c in cols:
        for lag in lags:
            df[f'{c}_lag_{lag}'] = df[c].shift(lag)
        for w in windows:
            df[f'{c}_roll_mean_{w}'] = df[c].rolling(w, min_periods=1).mean()
            df[f'{c}_roll_std_{w}'] = df[c].rolling(w, min_periods=1).std().fillna(0.0)
    return df

# build_feature_pipeline()
This is the main orchestrator that runs all the steps in order.

In [14]:
def build_feature_pipeline(weather_df,
                           latitude,
                           longitude,
                           system_kwp,
                           inverter_kw,
                           tilt,
                           azimuth,
                           noct=None,
                           albedo=0.2,
                           tz='UTC',
                           resample_freq='15T'):
    
    df = weather_df.copy()
    # ensure tz-aware index
    if df.index.tz is None:
        df.index = df.index.tz_localize(tz)

        df = ensure_frequency(df, freq=resample_freq)
    df = compute_solar_geometry(df, latitude, longitude, tz=tz)
    df = compute_poa(df, tilt=tilt, azimuth=azimuth, albedo=albedo)
    df = estimate_cell_temperature(df, mounting='open_rack', noct=noct)
    df = clear_sky_index(df, latitude, longitude, tz=tz)
    df = simple_dc_power(df, system_kwp)
    df = simple_inverter_ac(df, inverter_ac_rating_kw=inverter_kw)
    df = apply_system_losses(df)
    df = time_features(df)

    # add lag/rolling for key columns
    df = lag_and_rolling_features(df, cols=['p_ac_est_net', 'poa_global', 'ghi', 'k_clear'], lags=[1,2,3,4,8,24], windows=['1H','3H','6H','24H'])

    # drop rows with NaNs that cannot be used for model (first N rows)
    df = df.dropna()

    # choose feature columns (example)
    feature_cols = [
        'poa_global','poa_direct','poa_diffuse','ghi','dni','dhi','temp_air','wind_speed','cell_temp',
        'k_clear','hour_sin','hour_cos','doy_sin','doy_cos',
    ]
    # include lags/rolls
    feature_cols += [c for c in df.columns if any(x in c for x in ['lag','roll'])]

    X = df[feature_cols].copy()
    y = df['p_ac_est_net'].copy()  # if you have measured power, replace with measured target

    return X, y, df